In [1]:
import tensorflow as tf
import numpy as np
import scipy.misc
from tensorflow.keras.applications.resnet_v2 import ResNet50V2
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet_v2 import preprocess_input, decode_predictions
from tensorflow.keras import layers
from tensorflow.keras.layers import Input, Add, Dense, Activation, ZeroPadding2D,Flatten, Conv2D, AveragePooling2D, MaxPooling2D, GlobalMaxPooling2D, BatchNormalization
#from resnets_utils import *
from tensorflow.keras.initializers import random_uniform, glorot_uniform, constant, identity
from tensorflow.python.framework.ops import EagerTensor
from matplotlib.pyplot import imshow
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import *

In [2]:
classes = 6

# Building Model

In [3]:
class identity_block(layers.Layer):
  def __init__(self, filters, strides=1):
    super(identity_block, self).__init__()
    #self.conv1 = layers.Conv2D(filters, (1,1), strides=1, padding='valid',kernel_initializer=random_uniform(seed=0))
    self.conv1 = layers.Conv2D(filters, (1,1), strides=1, padding='valid',kernel_initializer=random_uniform)
    self.bn1 = layers.BatchNormalization(axis=3)
    self.relu = layers.Activation('relu')
    #self.conv2 = layers.Conv2D(filters, (3,3), strides=1, padding='same',kernel_initializer=random_uniform(seed=0))
    self.conv2 = layers.Conv2D(filters, (3,3), strides=1, padding='same',kernel_initializer=random_uniform)
    self.bn2 = layers.BatchNormalization(axis=3)
    #self.conv3 = layers.Conv2D(filters*4, (1,1), strides=1, padding='valid',kernel_initializer=random_uniform(seed=0))
    self.conv3 = layers.Conv2D(filters*4, (1,1), strides=1, padding='valid',kernel_initializer=random_uniform)
    self.bn3 = layers.BatchNormalization(axis=3)

  def call(self, inputs, training=True):
    out = self.conv1(inputs)
    out = self.bn1(out, training=training)
    out = self.relu(out)
    out = self.conv2(out)
    out = self.bn2(out, training=training)
    out = self.conv3(out)
    out = self.bn3(out)
    out = layers.add([out, inputs])
    out = self.relu(out)
    return out

In [4]:
class convolutional_block(layers.Layer):
  def __init__(self, filters, strides=1):
    super(convolutional_block, self).__init__()
    #self.conv1 = layers.Conv2D(filters, (1,1), strides=strides, padding='valid', kernel_initializer=glorot_uniform(seed=0))
    self.conv1 = layers.Conv2D(filters, (1,1), strides=strides, padding='valid', kernel_initializer=glorot_uniform)
    self.bn1 = layers.BatchNormalization(axis=3)
    self.relu = layers.Activation('relu')
    #self.conv2 = layers.Conv2D(filters, (3,3),strides=1 ,padding='same', kernel_initializer=glorot_uniform(seed=0))
    self.conv2 = layers.Conv2D(filters, (3,3),strides=1 ,padding='same', kernel_initializer=glorot_uniform)  
    self.bn2 = layers.BatchNormalization(axis=3)
    #self.conv3 = layers.Conv2D(filters*4, (1,1), strides=1, padding='valid', kernel_initializer=glorot_uniform)
    self.conv3 = layers.Conv2D(filters*4, (1,1), strides=1, padding='valid', kernel_initializer=glorot_uniform(seed=0))
    self.bn3 = layers.BatchNormalization(axis=3)
    self.downsample = tf.keras.Sequential()

    #self.downsample.add(layers.Conv2D(filters*4, (1,1), strides=strides, kernel_initializer=glorot_uniform(seed=0)))
    self.downsample.add(layers.Conv2D(filters*4, (1,1), strides=strides, kernel_initializer=glorot_uniform))
    self.downsample.add(layers.BatchNormalization(axis=3))

  def call(self, inputs, training = True):
    out = self.conv1(inputs)
    out = self.bn1(out, training = training)
    out = self.relu(out)
    out = self.conv2(out)
    out = self.bn2(out, training = training)
    out = self.relu(out)
    out = self.conv3(out)
    out = self.bn3(out, training = training)
    shortcut = self.downsample(inputs, training = training)
    #shortcut = layers.Conv2D(filters=out.shape[-1], kernel_size=(1,1), strides=1, padding='same', kernel_initializer=glorot_uniform(seed=0))(inputs)
    #shortcut = BatchNormalization(axis=3)(shortcut, training = training)
    out = layers.add([out, shortcut])
    out = self.relu(out)
    return out

In [5]:
class ResNet50(Model):
  def __init__(self,input_shape=(64,64,3)):
    super(ResNet50,self).__init__()

    self.conv_1 = tf.keras.Sequential([
        ZeroPadding2D((3,3)),
        #Conv2D(64, (7,7), strides=2, kernel_initializer=glorot_uniform(seed=0)),
        Conv2D(64, (7,7), strides=2, kernel_initializer=glorot_uniform),
        BatchNormalization(axis=3),
        Activation('relu'),
        MaxPooling2D((3,3), strides=2),
    ])

    self.block2 = tf.keras.Sequential([
        #convolutional_block(f=3, filters=[64,64,256], s=1),
        convolutional_block(filters=64),
        identity_block(filters=64),
        identity_block(filters=64)
    ])

    self.block3 = tf.keras.Sequential([
        #convolutional_block(f=3, filters=[128,128,512],s=2),
        convolutional_block(filters=128,strides=2),
        identity_block(filters=128),
        identity_block(filters=128),
        identity_block(filters=128)
    ])

    self.block4 = tf.keras.Sequential([
        #convolutional_block(f=3, filters=[256,256,1024],s=2),
        convolutional_block(filters=256,strides=2),
        identity_block(filters=256),
        identity_block(filters=256),
        identity_block(filters=256),
        identity_block(filters=256),
        identity_block(filters=256),
    ])

    self.block5 = tf.keras.Sequential([
        #convolutional_block(f=3,filters=[512,512,2048],s=2),
        convolutional_block(filters=512,strides=2),
        identity_block(filters=512),
        identity_block(filters=512),
    ])

    self.fc = tf.keras.Sequential([
        AveragePooling2D(pool_size=(2,2)),
        Flatten(),
        #Dense(6, activation='softmax', kernel_initializer=glorot_uniform(seed=0))
        Dense(6, activation='softmax', kernel_initializer=glorot_uniform)
    ])
  def call(self, inputs, training=False):
    x = self.conv_1(inputs)
    x = self.block2(x)
    x = self.block3(x)
    x = self.block4(x)
    x = self.block5(x)
    return self.fc(x)



# Loading Dataset

In [6]:
import h5py
def load_dataset():
    train_dataset = h5py.File('./datasets/Files/train_signs.h5', "r")
    
    train_set_x_orig = np.array(train_dataset["train_set_x"][:])
    train_set_y_orig = np.array(
        train_dataset["train_set_y"][:])  

    test_dataset = h5py.File('./datasets/Files/test_signs.h5', "r")
    
    test_set_x_orig = np.array(test_dataset["test_set_x"][:])
    test_set_y_orig = np.array(
        test_dataset["test_set_y"][:])  

    classes = np.array(test_dataset["list_classes"][:])  

    train_set_y_orig = train_set_y_orig.reshape((1, train_set_y_orig.shape[0]))
    test_set_y_orig = test_set_y_orig.reshape((1, test_set_y_orig.shape[0]))

    return train_set_x_orig, train_set_y_orig, test_set_x_orig, test_set_y_orig, classes

In [7]:
X_train_orig, y_train_orig, X_test_orig, y_test_orig, classes = load_dataset()

In [8]:
y_train_orig.shape

(1, 1080)

In [9]:
y_train_reshape = y_train_orig.reshape((y_train_orig.shape[1],))

In [10]:
y_train_reshape

array([5, 0, 2, ..., 2, 4, 5], dtype=int64)

In [11]:
X_train = X_train_orig / 255.
X_test = X_test_orig /255.

y_train = tf.keras.utils.to_categorical(y_train_orig, num_classes=len(classes))
y_test = tf.keras.utils.to_categorical(y_test_orig, num_classes=len(classes))

print(f'y_train shape: {y_train.shape}')
print(f'y_test shape: {y_test.shape}')

y_train shape: (1, 1080, 6)
y_test shape: (1, 120, 6)


In [12]:
X_train.shape

(1080, 64, 64, 3)

In [13]:
y_train = y_train.reshape((y_train.shape[1], y_train.shape[2]))
y_test = y_test.reshape((y_test.shape[1], y_test.shape[2]))

In [14]:
print(f'X_train shape: {X_train.shape}')
print(f'X_test shape: {X_test.shape}')
print(f'y_train shape: {y_train.shape}')
print(f'y_test shape: {y_test.shape}')

X_train shape: (1080, 64, 64, 3)
X_test shape: (120, 64, 64, 3)
y_train shape: (1080, 6)
y_test shape: (120, 6)


# Training Model

In [16]:
keras.backend.set_learning_phase(True)
model = ResNet50()

In [17]:
model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy, optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), metrics=['accuracy'])

In [18]:
model.fit(X_train, y_train_reshape, epochs=5,shuffle=True)

Epoch 1/5

34/34 ━━━━━━━━━━━━━━━━━━━━ 166s 3s/step - accuracy: 0.4237 - loss: 2.1965
Epoch 2/5
34/34 ━━━━━━━━━━━━━━━━━━━━ 92s 3s/step - accuracy: 0.8008 - loss: 0.5267
Epoch 3/5
34/34 ━━━━━━━━━━━━━━━━━━━━ 144s 3s/step - accuracy: 0.8995 - loss: 0.3149
Epoch 4/5
34/34 ━━━━━━━━━━━━━━━━━━━━ 144s 3s/step - accuracy: 0.9018 - loss: 0.2763
Epoch 5/5
34/34 ━━━━━━━━━━━━━━━━━━━━ 136s 3s/step - accuracy: 0.9649 - loss: 0.1338


In [21]:
tf.keras.backend.set_learning_phase(False)
model.evaluate(X_train, y_train_reshape)

AttributeError: module 'keras._tf_keras.keras.backend' has no attribute 'set_learning_phase'

In [ ]:
from sklearn.metrics import classification_report